In [1]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

fatal: destination path 'detectron2' already exists and is not an empty directory.


In [2]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger


setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from imutils import paths
import os

# Run a pre-trained detectron2 model

We first download an image from the COCO dataset:

In [4]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

[08/17 12:16:22 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


In [ ]:

Original_data_set = "/content/drive/MyDrive/Thesis/Original_pictures/IndividualID/sociable_weavers/Val/011016A9DF"
imagePaths = sorted(list(paths.list_images(Original_data_set)))
for i in range(0,len(imagePaths)):
  image_path = imagePaths[i]
  if not image_path.endswith('.jpg'):
        continue
  image = cv2.imread(image_path)
  if image is None:
      continue
  if image.shape[0] == 0 or image.shape[1] == 0:
    continue
  outputs = predictor(image)
  instances = outputs["instances"]
  if not instances:
    continue
  bird_class_idx = predictor.metadata.thing_classes.index("bird")
  bird_instances = instances[instances.pred_classes == bird_class_idx]
  if not bird_instances:
        continue
  bird_instances=bird_instances[0]

  bird_mask = outputs["instances"].pred_masks[0].cpu().numpy()
  # cv2_imshow(bird_mask)
  image_np = np.array(image)
  for box, mask in zip(bird_instances.pred_boxes, bird_instances.pred_masks):
      x_min, y_min, x_max, y_max = box.cpu().numpy()
      x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)

      # Draw bounding boxes
      image_np=cv2.rectangle(image_np, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
      cv2_imshow(image_np)
      # Draw masks
    # mask_np = mask.cpu().numpy()
    # image_np[mask_np] = np.array([0, 0, 0])
      image_np = image_np[y_min:y_max, x_min:x_max]
  cropped_image = image_np
  # Convert bird mask to binary mask (0s for background, 1s for bird)
  binary_mask = bird_mask.astype(np.uint8)
  cv2_imshow(binary_mask)
  # Invert the binary mask (0s for bird, 1s for background)
  binary_mask_inv = 1 - binary_mask
  cv2_imshow(binary_mask_inv)
  # Convert image to numpy array
  cropped_image = np.array(image)

  # Set the pixels outside the bird mask to black
  cropped_image[binary_mask_inv == 1] = 0

  # Draw bounding boxes and masks for each bird instance
  for box, mask in zip(bird_instances.pred_boxes, bird_instances.pred_masks):
      x_min, y_min, x_max, y_max = box.cpu().numpy()
      x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)

      # Draw bounding boxes
      cv2.rectangle(cropped_image, (x_min, y_min), (x_max, y_max), (0, 0, 0), 2)

      # Draw masks
      mask_np = mask.cpu().numpy()
      cropped_image[mask_np] = np.array([0, 255, 0])
      # stencil = np.zeros(image_np.shape).astype(image_np.dtype)
      # color = [255, 255, 255]
      # Crop the image to the bird region
      image_np_cropped = cropped_image[y_min:y_max, x_min:x_max]
  cropped_image = image_np_cropped

  # Convert the cropped image to grayscale
  cropped_gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
  cv2_imshow(cropped_gray)
  # Threshold the grayscale image to create a binary mask
  _, bird_mask = cv2.threshold(cropped_gray, 1, 255, cv2.THRESH_BINARY)

  # Use the binary mask to extract the bird region from the original image
  original_image = image_np
  final_image = cv2.bitwise_and(original_image, original_image, mask=bird_mask)
  cv2_imshow(final_image)
  # new_folder_path = '/content/drive/MyDrive/Thesis/Detectron_cropped_images/IndividualID/sociable_weavers/Val/0700EE42FD'
  # if not os.path.exists(new_folder_path):
  #   os.makedirs(new_folder_path)
  # cv2.imwrite("/content/drive/MyDrive/Thesis/Detectron_cropped_images/IndividualID/sociable_weavers/Val/0700EE42FD/"+str(i)+"_"+imagePaths[i].split("/")[-1:][0], final_image)



In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
